In [ ]:
import sys; sys.path.insert(0, '..')
import findspark; findspark.init()

In [1]:
import json
import configparser
from os import environ, listdir, path

from pyspark import SparkConf
from pyspark import SparkFiles
from pyspark.sql import SparkSession

from etl_project.commons import utils

In [2]:
# EMR 6.10
environ['PYSPARK_SUBMIT_ARGS'] = "--packages=com.amazonaws:aws-java-sdk:1.11.900,org.apache.hadoop:hadoop-aws:3.2.0 pyspark-shell"

In [3]:
environ['DEBUG'] = "1"

In [4]:
config = configparser.ConfigParser()
config.read(path.expanduser("~/.aws/credentials"))
access_id = config.get('default', "aws_access_key_id") 
access_key = config.get('default', "aws_secret_access_key")

In [5]:
session, logger= utils.start_spark()

In [6]:
# https://hadoop.apache.org/docs/current3/hadoop-aws/tools/hadoop-aws/s3n.html#How_to_migrate_to_to_the_S3A_client
hadoop_conf=session._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", access_id)
hadoop_conf.set("fs.s3a.secret.key", access_key)

In [11]:
df=session.read.csv("s3a://covid19-lake/static-datasets/csv/state-abv/states_abv.csv", header=True)

In [14]:
df.write.parquet("s3a://pyspark3-sample/states.parquet",mode="overwrite")

In [ ]:
session.stop()